In [1]:
import nannyml as nml
import numpy as np
import pandas as pd

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# Make pandas display all columns and rows
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [3]:


def generate_multiclass_data(num_samples, num_features, confusion_matrix):
    num_classes = len(confusion_matrix)
    
    # Generate feature data
    X = np.random.rand(num_samples, num_features)
    
    # Generate true classes
    true_classes = np.random.choice(num_classes, num_samples, p=[0.33, 0.33, 0.34])
    true_classes = np.array(['a', 'b', 'c'])[true_classes]
    
    # Generate predicted classes and probabilities based on the confusion matrix
    predicted_classes = []
    predicted_probs = []

    for true_class in true_classes:
        idx = np.where(np.array(['a', 'b', 'c']) == true_class)[0][0]
        predicted_class = np.random.choice(num_classes, 1, p=confusion_matrix[idx])[0]
        predicted_probs.append(confusion_matrix[idx])
        
        predicted_classes.append(np.array(['a', 'b', 'c'])[predicted_class])

    return X, true_classes, predicted_classes, predicted_probs

# Define the confusion matrix
confusion_matrix = [
    [0.7, 0.2, 0.1],  # True class 'a'
    [0.1, 0.8, 0.1],  # True class 'b'
    [0.2, 0.2, 0.6],  # True class 'c'
]

num_samples = 1000
num_features = 5

# Generate data
X, true_classes, predicted_classes, predicted_probs = generate_multiclass_data(num_samples, num_features, confusion_matrix)

# Create reference and analysis DataFrames
reference = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(num_features)])
reference['true_class'] = true_classes
reference['predicted_class'] = predicted_classes
reference[['predicted_prob_a', 'predicted_prob_b', 'predicted_prob_c']] = predicted_probs

X2, true_classes2, predicted_classes2, predicted_probs2 = generate_multiclass_data(num_samples, num_features, confusion_matrix)
analysis = pd.DataFrame(X2, columns=[f'feature_{i}' for i in range(num_features)])
analysis['true_class'] = true_classes2
analysis['predicted_class'] = predicted_classes2
analysis[['predicted_prob_a', 'predicted_prob_b', 'predicted_prob_c']] = predicted_probs2


In [4]:

analysis.head(3)


,feature_0,feature_1,feature_2,feature_3,feature_4,true_class,predicted_class,predicted_prob_a,predicted_prob_b,predicted_prob_c
0,0.806663,0.619435,0.980898,0.640137,0.700309,b,b,0.1,0.8,0.1
1,0.451920,0.594596,0.892019,0.368459,0.942581,c,a,0.2,0.2,0.6
2,0.009097,0.480491,0.739903,0.069814,0.969340,c,b,0.2,0.2,0.6


# Calc

In [5]:
calc = nml.PerformanceCalculator(
    y_pred_proba={
        'a': 'predicted_prob_a',
        'b': 'predicted_prob_b',
        'c': 'predicted_prob_c',
    },
    y_pred='predicted_class',
    y_true='true_class',
    problem_type='classification_multiclass',
    metrics=['confusion_matrix'],
    normalize_confusion_matrix='true'
)

In [6]:
calc.fit(reference)

In [7]:
results = calc.calculate(analysis)

In [8]:
figure = results.plot()
figure.show()

# Estimation

In [9]:
# make a copy of the analysis data and drop the y_true column
analysis2 = analysis.copy()
analysis2.drop('true_class', axis=1, inplace=True)

In [10]:
estimator = nml.CBPE(
    y_pred_proba={
        'a': 'predicted_prob_a',
        'b': 'predicted_prob_b',
        'c': 'predicted_prob_c',
    },
    y_pred='predicted_class',
    y_true='true_class',
    problem_type='classification_multiclass',
    metrics=['confusion_matrix'],
    normalize_confusion_matrix='all'
)

In [11]:
reference.head(3)

,feature_0,feature_1,feature_2,feature_3,feature_4,true_class,predicted_class,predicted_prob_a,predicted_prob_b,predicted_prob_c
0,0.742410,0.992336,0.197242,0.761573,0.959133,b,b,0.1,0.8,0.1
1,0.775063,0.408820,0.626745,0.526839,0.800511,c,a,0.2,0.2,0.6
2,0.943655,0.483067,0.551708,0.710890,0.588761,c,b,0.2,0.2,0.6


In [12]:
estimator.fit(reference)

In [13]:
results = estimator.estimate(analysis)

In [14]:
results.data

chunk                                                        \
          key chunk_index start_index end_index start_date end_date   
0      [0:99]           0           0        99       None     None   
1   [100:199]           1         100       199       None     None   
2   [200:299]           2         200       299       None     None   
3   [300:399]           3         300       399       None     None   
4   [400:499]           4         400       499       None     None   
5   [500:599]           5         500       599       None     None   
6   [600:699]           6         600       699       None     None   
7   [700:799]           7         700       799       None     None   
8   [800:899]           8         800       899       None     None   
9   [900:999]           9         900       999       None     None   
10     [0:99]           0           0        99       None     None   
11  [100:199]           1         100       199       None     None   
12  [200:299]           2         200       299       None     None   
13  [300:399]           3         300       399       None     None   
14  [400:499]           4         400       499       None     None   
15  [500:599]           5         500       599       None     None   
16  [600:699]           6         600       699       None     None   
17  [700:799]           7         700       799       None     None   
18  [800:899]           8         800       899       None     None   
19  [900:999]           9         900       999       None     None   

              true_a_pred_a                                                    \
       period         value sampling_error realized upper_confidence_boundary   
0   reference         0.189       0.042147     0.24                  0.315442   
1   reference         0.159       0.042147     0.22                  0.285442   
2   reference         0.153       0.042147     0.19                  0.279442   
3   reference         0.119       0.042147     0.15                  0.245442   
4   reference         0.187       0.042147     0.24                  0.313442   
5   reference         0.168       0.042147     0.22                  0.294442   
6   reference         0.166       0.042147     0.22                  0.292442   
7   reference         0.186       0.042147     0.24                  0.312442   
8   reference         0.233       0.042147     0.30                  0.359442   
9   reference         0.220       0.042147     0.29                  0.346442   
10   analysis         0.170       0.042147     0.21                  0.296442   
11   analysis         0.224       0.042147     0.30                  0.350442   
12   analysis         0.191       0.042147     0.25                  0.317442   
13   analysis         0.224       0.042147     0.29                  0.350442   
14   analysis         0.168       0.042147     0.21                  0.294442   
15   analysis         0.195       0.042147     0.25                  0.321442   
16   analysis         0.174       0.042147     0.23                  0.300442   
17   analysis         0.139       0.042147     0.18                  0.265442   
18   analysis         0.179       0.042147     0.22                  0.305442   
19   analysis         0.153       0.042147     0.19                  0.279442   

                                                                     \
   lower_confidence_boundary upper_threshold lower_threshold  alert   
0                   0.062558         0.35502         0.10698  False   
1                   0.032558         0.35502         0.10698  False   
2                   0.026558         0.35502         0.10698  False   
3                   0.000000         0.35502         0.10698  False   
4                   0.060558         0.35502         0.10698  False   
5                   0.041558         0.35502         0.10698  False   
6                   0.039558         0.35502         0.10698  False   
7                   0.0

In [15]:
metric_fig = results.plot()
metric_fig.show()

# Estimation with NML data

In [16]:
reference_df, analysis_df, analysis_target_df = nml.load_synthetic_multiclass_classification_dataset()
analysis_df = analysis_df.merge(analysis_target_df, left_index=True, right_index=True)

In [17]:
estimator = nml.CBPE(
    y_pred_proba={
        'prepaid_card': 'y_pred_proba_prepaid_card',
        'highstreet_card': 'y_pred_proba_highstreet_card',
        'upmarket_card': 'y_pred_proba_upmarket_card'},
    y_pred='y_pred',
    y_true='y_true',
    timestamp_column_name='timestamp',
    problem_type='classification_multiclass',
    metrics=['confusion_matrix'],
    chunk_size=1000,
    normalize_confusion_matrix='all'
)

calc = nml.PerformanceCalculator(
    y_pred_proba={
        'prepaid_card': 'y_pred_proba_prepaid_card',
        'highstreet_card': 'y_pred_proba_highstreet_card',
        'upmarket_card': 'y_pred_proba_upmarket_card'},
    y_pred='y_pred',
    y_true='y_true',
    timestamp_column_name='timestamp',
    problem_type='classification_multiclass',
    metrics=['confusion_matrix'],
    chunk_size=1000,
    normalize_confusion_matrix='all'
)

In [18]:
estimator.fit(reference_df)
calc.fit(reference_df)

In [19]:
estimated_results = estimator.estimate(analysis_df)
calculated_results = calc.calculate(analysis_df)

In [20]:
fig1 = estimated_results.plot()
fig1.show()

In [21]:
fig2 = calculated_results.plot()
fig2.show()

In [22]:
calculated_results.data

chunk                                                        \
               key chunk_index start_index end_index          start_date   
0          [0:999]           0           0       999 2020-05-02 02:01:30   
1      [1000:1999]           1        1000      1999 2020-05-04 05:34:39   
2      [2000:2999]           2        2000      2999 2020-05-06 09:33:22   
3      [3000:3999]           3        3000      3999 2020-05-08 12:04:36   
4      [4000:4999]           4        4000      4999 2020-05-10 11:54:56   
5      [5000:5999]           5        5000      5999 2020-05-12 13:14:18   
6      [6000:6999]           6        6000      6999 2020-05-14 12:29:25   
7      [7000:7999]           7        7000      7999 2020-05-16 10:55:36   
8      [8000:8999]           8        8000      8999 2020-05-18 12:03:56   
9      [9000:9999]           9        9000      9999 2020-05-20 12:35:56   
10   [10000:10999]          10       10000     10999 2020-05-22 13:41:07   
11   [11000:11999]          11       11000     11999 2020-05-24 17:21:32   
12   [12000:12999]          12       12000     12999 2020-05-26 18:31:06   
13   [13000:13999]          13       13000     13999 2020-05-28 17:17:31   
14   [14000:14999]          14       14000     14999 2020-05-30 15:45:32   
15   [15000:15999]          15       15000     15999 2020-06-01 18:59:40   
16   [16000:16999]          16       16000     16999 2020-06-03 20:13:13   
17   [17000:17999]          17       17000     17999 2020-06-05 19:10:39   
18   [18000:18999]          18       18000     18999 2020-06-07 19:58:39   
19   [19000:19999]          19       19000     19999 2020-06-09 19:16:48   
20   [20000:20999]          20       20000     20999 2020-06-11 19:55:34   
21   [21000:21999]          21       21000     21999 2020-06-13 19:22:04   
22   [22000:22999]          22       22000     22999 2020-06-15 19:08:03   
23   [23000:23999]          23       23000     23999 2020-06-17 20:22:19   
24   [24000:24999]          24       24000     24999 2020-06-19 19:44:14   
25   [25000:25999]          25       25000     25999 2020-06-21 20:51:22   
26   [26000:26999]          26       26000     26999 2020-06-23 22:06:03   
27   [27000:27999]          27       27000     27999 2020-06-25 23:29:01   
28   [28000:28999]          28       28000     28999 2020-06-27 23:51:02   
29   [29000:29999]          29       29000     29999 2020-06-30 00:15:57   
30   [30000:30999]          30       30000     30999 2020-07-02 02:06:56   
31   [31000:31999]          31       31000     31999 2020-07-04 04:21:19   
32   [32000:32999]          32       32000     32999 2020-07-06 05:09:22   
33   [33000:33999]          33       33000     33999 2020-07-08 05:58:24   
34   [34000:34999]          34       34000     34999 2020-07-10 05:56:40   
35   [35000:35999]          35       35000     35999 2020-07-12 05:35:07   
36   [36000:36999]          36       36000     36999 2020-07-14 08:14:08   
37   [37000:37999]          37       37000     37999 2020-07-16 08:28:16   
38   [38000:38999]          38       38000     38999 2020-07-18 06:40:12   
39   [39000:39999]          39       39000     39999 2020-07-20 09:36:36   
40   [40000:40999]          40       40000     40999 2020-07-22 12:58:29   
41   [41000:41999]          41       41000     41999 2020-07-24 12:35:02   
42   [42000:42999]          42       42000     42999 2020-07-26 12:57:37   
43   [43000:43999]          43       43000     43999 2020-07-28 09:29:44   
44   [44000:44999]          44       44000     44999 2020-07-30 06:54:08   
45   [45000:45999]          45       45000     45999 2020-08-01 10:16:21   
46   [46000:46999]          46       46000     46999 2020-08-03 11:12:47   
47   [47000:47999]          47       47000     47999 2020-08-05 12:38:10   
48   [48000:48999]          48       48000     48999 2020-08-07 16:33:44   
49   [49000:49999]          49       49000     49999 2020-08-09 18:15:12   
50   [50000:50999]          50       50000     50999 2020-08-